In [ ]:
import random
import pandas as pd
import vk_api 
from vk_api.longpoll import VkLongPoll, VkEventType
from vk_api.utils import get_random_id
from vk_api.keyboard import VkKeyboard, VkKeyboardColor

In [45]:
group_key = 'vk1.a.IozVEQToDm3N8sOnFPt7xecrQTGnmCHM-BeOuM0gl-PcbmJxqtw4P1r-wa7rkno55vyQTu-2yMvDu6etw11kq7WQIf7cEon_Q9trQp5uwIsejUgw0VVh2yZWNB9SiIYjXJ72EBY_rgwDOYb5L3qM8ojj6wQDdoXO81qak5HH8J3CmPpN8u3167TxIB7lpeZP2OzYyO97QNhCg7d6T8PA8A'
vk_session = vk_api.VkApi(token = group_key)
longpoll = VkLongPoll(vk_session)
vk = vk_session.get_api()

In [119]:
# Отправляем статистику пользователя
def check_stats(user_id, user_stats=user_stats):
    user_stat = user_stats.get(user_id)
    if user_stat['number_of_all_answers'] != 0:
        message = f"""Статистика ваших ответов:
            Правильных: {user_stat['number_of_right_answers']}
            Всего: {user_stat['number_of_all_answers']}
            Процент: {user_stat['number_of_right_answers']/user_stat['number_of_all_answers'] * 100}
        """
        vk.messages.send(
            user_id=user_id,
            message=message,
            random_id=0
        )
    else:
        vk.messages.send(
            user_id=user_id,
            message='Вы еще не прошли ни одной викторины',
            random_id=0
        )

In [125]:
quiz_files = {
    'свойства воздуха': 'свойства воздуха.csv',
}

materials = pd.read_csv('data/материалы.csv', delimiter=';')

# Настройки клавиатуры
keyboard = VkKeyboard(one_time=False)
keyboard.add_button('свойства воздуха', color=VkKeyboardColor.SECONDARY)
keyboard.add_button('статистика', color=VkKeyboardColor.POSITIVE)

# Структура (пользователь -> статистика)
user_stats = {}

for event in longpoll.listen():
    if event.type == VkEventType.MESSAGE_NEW and event.to_me:
        user_id = event.user_id
        text = event.text.lower()
        
        # Переменные для подсчета статистики
        if not user_id in user_stats.keys(): 
            user_stats[user_id] = {
                'number_of_right_answers': 0,
                'number_of_all_answers': 0
            }
        
        if text == 'привет':           
            vk.messages.send(
                user_id=user_id,
                message="Выберите викторину:",
                random_id=0,
                keyboard=keyboard.get_keyboard()
            )
            
        elif text in quiz_files.keys():
            # Пользователь выбрал викторину, начинаем вопросы
            quiz_name = text
            df = pd.read_csv(f'data/{quiz_files[quiz_name]}', encoding='utf-8', delimiter=';')
            all_questions = df['формулировка вопроса'].unique()
            for question in all_questions:
                possible_answers = list(df[df['формулировка вопроса']==question]['вариант ответа'])
                right_answer_text = df[df['формулировка вопроса']==question] \
                   [df['правильность']==1]['вариант ответа'].item()
                right_answer_index = possible_answers.index(right_answer_text) + 1
                
                message = f'''{question} 
                1. {possible_answers[0]} 
                2. {possible_answers[1]}
                3. {possible_answers[2]}
                4. {possible_answers[3]}'''
                
                vk.messages.send(
                        user_id=user_id,
                        message=message,
                        random_id=0
                    )
                is_right_answer = True
                
                for answer_event in longpoll.listen():
                    if answer_event.type == VkEventType.MESSAGE_NEW and answer_event.to_me:
                        user_answer = int(answer_event.text)
                        
                        if user_answer not in (1, 2, 3, 4):
                            error_message = 'неправильный формат ответа. ответ должен быть цифрой от 1 до 4'
                            vk.messages.send(
                                user_id=user_id,
                                message=error_message,
                                random_id=0
                            )
                        user_stats[user_id]['number_of_all_answers'] += 1
                        
                        if user_answer != right_answer_index:
                            is_right_answer = False
                        break
                            
                if not is_right_answer:
                    material_message = materials[materials['название темы']==quiz_name]['ссылка на материал'].item()
                    vk.messages.send(
                                user_id=user_id,
                                message=f'ответ неверный. ссылка для повторения: {material_message}',
                                random_id=0
                            )
                    break
                else:
                    user_stats[user_id]['number_of_right_answers'] += 1
                    vk.messages.send(
                                user_id=user_id,
                                message=f'ответ верный. следующий вопрос',
                                random_id=0
                            )
                        
        elif text == 'статистика':
            # Отправляем статистику пользователя
            check_stats(user_id, user_stats)
        else: 
            vk.messages.send(
                    user_id=user_id,
                    message='Неизвестная команда',
                    random_id=0
                )

/var/folders/f5/mpttzb9n0f99d4gzb95z0k8r0000gn/T/ipykernel_66847/654302172.py:42: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  right_answer_text = df[df['формулировка вопроса']==question] \


KeyboardInterrupt: 

In [67]:
df = pd.read_csv('data/свойства воздуха.csv', delimiter=';')

In [122]:
user_stats.get(101180492)['number_of_all_answers']

2